In [1]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/samsung_data_set.csv',encoding='cp949')

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# df['content'] = df['title']

# title 열에 대해 형태소 분석 적용
df['title'] = df['title'].apply(tokenize)

# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'


df['title'] = df['title'].apply(tokenize)

df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 1


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: phrase input should be string, not <class 'list'>

In [ ]:
df.to_csv(f'csv/samsung_train.csv', index=False)